# MIDI file cutter
>cuts midi files in 1 bar sequences and writes to disk <br>
__CAUTION__: You have to manually create the folder, where you want to place the cut sequences, I used /sequences

In [1]:
import numpy as np
import glob
import time
import os
import pypianoroll as ppr
from utils.utilsPreprocessing import *

In [2]:
beat_res = 24
seq_length = 96

In [3]:
def cut_single_track(track):
    too_long = track.shape[0]%seq_length
    if(too_long != 0):
        track = track[:-too_long,:]
    sequences = np.array(np.vsplit(track, int(track.shape[0]/seq_length)))
    return sequences

def write_to_disk(sequences, path):
    for i, seq in enumerate(sequences):
        if(np.any(seq)):
            tempTrack = ppr.Track(seq)
            newTrack = ppr.Multitrack()
            newTrack.append_track(tempTrack)
            newTrack.write(os.path.dirname(path)+'/sequences/'+os.path.splitext(os.path.basename(path))[0]+'{}'.format(i))
        else:
            continue

In [4]:
path_to_files = glob.glob('/Volumes/EXT/DATASETS/test/*.mid')

for i, path in enumerate(path_to_files):
    track = ppr.Multitrack(path, beat_resolution=beat_res)
    track = track.get_stacked_pianoroll()
    
    #single track midi file
    if(track.shape[2] == 1):
        track = np.squeeze(track,2)
        sequences = cut_single_track(track)
        write_to_disk(sequences, path)
        
    #multitrack midi file
    else:
        track = np.rollaxis(track,2,0)
        for track_number,instrument_track in enumerate(track):
            temp_path = os.path.splitext(path)[0]+'_'+str(track_number)+'_'+os.path.splitext(path)[1]
            sequences = cut_single_track(instrument_track)
            write_to_disk(sequences, temp_path)
    
    print('{}/{} tracks have been cut'.format(i+1,len(path_to_files)))

/Users/nw/Uni/Midi_Parser/.env3/lib/python3.6/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


0/2 tracks have been cut
1/2 tracks have been cut
